In [1]:
%load_ext autoreload
%autoreload 2
import json
import sys
import os
import pandas as pd
from math import floor
import numpy as np
import pickle
sys.path.append(os.path.abspath('../../../utils'))
from protocol_system_compiler import *

In [2]:
fluids = pd.read_csv('fluids.csv')

In [3]:
fluids

,NodeID,FluidType,BufferName,LoadVolume
0,ot2,source,ot2,3
1,2,source,wash buffer,40
2,3,source,imaging buffer,21
3,4,source,tcep,20
4,6,source,2x-SSC,50
5,7,source,universal_fluors,15
6,waste,destination,waste_tank,0


In [4]:
labware = {'tips_200':{
    'api_name': 'opentrons_96_filtertiprack_200ul',
    'location': '1',
    'creator': 'opentrons'
},
           'tips_1000':{
    'api_name': 'opentrons_96_filtertiprack_1000ul',
    'location': '3',
    'creator': 'opentrons'
},
              'reservoir1':{
    'api_name': 'nest_12_reservoir_15ml',
    'location': '5',
    'creator': 'opentrons'
                },
              'wellplate1':{
    'api_name': 'nest_96_wellplate_2ml_deep',
    'location': '7',
    'creator': 'opentrons'
                },
              'syringe_adapter':{
    'api_name': 'yeolab_1_bd_syringe_adap_3000ul',
    'location': '10',
    'creator': 'custom'
                },
              }
hardware = {'p200':{
    'api_name': 'p300_single_gen2',
    'mount': 'left',
    'tip_racks': ['tips_200']},
            'p1000':{
    'api_name': 'p1000_single_gen2',
    'mount': 'right',
    'tip_racks': ['tips_1000']}}

metadata = {'apiLevel': '2.14'}

ot2_config = {'labware':labware,
              'hardware':hardware,
              'metadata':metadata}

In [5]:
def get_fluid_node(buffer):
    return fluids[fluids['BufferName']==buffer]['NodeID'].iloc[0]

In [6]:
reagents = {}
def generate_well_pos(well_num,total=96,ncol=12,nrow=8):
    row = well_num//ncol
    col = well_num%ncol
    return chr(65+row)+str(col+1)
num_rounds = 10
for i in range(num_rounds):
    reagents['round'+str(i+1)] = {'labware':'wellplate1',
                                'location':generate_well_pos(i),
                                'volume':750,
                           'hardware':'p1000',
                           'repeat':2,
                           'rate':0.5,
                           'wait':3}
# Add adapter wash buffer for flushing flow cell + flow line from OT-2
for i in range(num_rounds):
    reagents['adap_wash_'+str(i+1)+'_1'] = {'labware':'wellplate1',
                                            'location':f'C{i+1}',
                                            'volume':1000,
                                            'hardware':'p1000',
                                            'repeat':1,
                                            'rate':0.5,
                                            'wait':3}
    reagents['adap_wash_'+str(i+1)+'_2'] = {'labware':'wellplate1',
                                            'location':f'D{i+1}',
                                            'volume':1000,
                                            'hardware':'p1000',
                                            'repeat':1,
                                            'rate':0.5,
                                            'wait':3}

In [7]:
reagents['adap_wash_7_2']

{'labware': 'wellplate1',
 'location': 'D7',
 'volume': 1000,
 'hardware': 'p1000',
 'repeat': 1,
 'rate': 0.5,
 'wait': 3}

In [8]:
step_n = 0
experiment = {}
global_pump_speed = 0.2 # temp work around since sample appears very viscous??


for round_n in range(1,11):
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'liquid_handler'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['reagent'] = f'round{round_n}'
    experiment[step_n]['step_metadata']['info'] = reagents[f'round{round_n}']
    experiment[step_n]['step_metadata']['destination'] = 'syringe_adapter'
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('ot2')
    experiment[step_n]['step_metadata']['volume'] = 1.5
    experiment[step_n]['step_metadata']['speed'] = global_pump_speed
    experiment[step_n]['step_metadata']['pump_wait'] = 60
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'wait'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['wait_time'] = 15*60 # 15 minutes
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'liquid_handler'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['reagent'] = f'adap_wash_{round_n}_1'
    experiment[step_n]['step_metadata']['info'] = reagents[f'adap_wash_{round_n}_1']
    experiment[step_n]['step_metadata']['destination'] = 'syringe_adapter'
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'liquid_handler'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['reagent'] = f'adap_wash_{round_n}_2'
    experiment[step_n]['step_metadata']['info'] = reagents[f'adap_wash_{round_n}_2']
    experiment[step_n]['step_metadata']['destination'] = 'syringe_adapter'
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('ot2')
    experiment[step_n]['step_metadata']['volume'] = 2.0
    experiment[step_n]['step_metadata']['speed'] = global_pump_speed
    experiment[step_n]['step_metadata']['pump_wait'] = 60
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('universal_fluors')
    experiment[step_n]['step_metadata']['volume'] = 1.5
    experiment[step_n]['step_metadata']['speed'] = global_pump_speed
    experiment[step_n]['step_metadata']['pump_wait'] = 60
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'wait'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['wait_time'] = 15*60 # 15 minutes
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('wash buffer')
    experiment[step_n]['step_metadata']['volume'] = 2
    experiment[step_n]['step_metadata']['speed'] = global_pump_speed
    experiment[step_n]['step_metadata']['pump_wait'] = 60
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('imaging buffer')
    experiment[step_n]['step_metadata']['volume'] = 1.5
    experiment[step_n]['step_metadata']['speed'] = global_pump_speed
    experiment[step_n]['step_metadata']['pump_wait'] = 60
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'wait'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['wait_time'] = 5*60 # 5 minutes
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'image'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['filename'] = f'round_{round_n}'
    experiment[step_n]['step_metadata']['callibrate_af'] = True
    experiment[step_n]['slack_notify'] = True
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('tcep')
    experiment[step_n]['step_metadata']['volume'] = 2
    experiment[step_n]['step_metadata']['speed'] = global_pump_speed
    experiment[step_n]['step_metadata']['pump_wait'] = 60
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'wait'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['wait_time'] = 5*60 # 5 minutes
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('2x-SSC')
    experiment[step_n]['step_metadata']['volume'] = 5
    experiment[step_n]['step_metadata']['speed'] = global_pump_speed
    experiment[step_n]['step_metadata']['pump_wait'] = 60
    experiment[step_n]['slack_notify'] = False
    step_n += 1

In [9]:
list(experiment.keys())[-10:]

[140, 141, 142, 143, 144, 145, 146, 147, 148, 149]

In [10]:
experiment[146]

{'step_type': 'image',
 'step_metadata': {'filename': 'round_10', 'callibrate_af': True},
 'slack_notify': True}

In [11]:
del experiment[147]
del experiment[148]
del experiment[149]

In [12]:
with open('experiment.json', 'w+') as f:
    json.dump(experiment, f, indent=4)


In [13]:
with open('ot2_config.json', 'w+') as f:
    json.dump(ot2_config, f, indent=4)


In [2]:
with open('C:\\Users\\ONI\\Dropbox\\Yeo lab\\Noor\\gabeen\\runs\\MR-012-final-positions.pkl','rb') as f:
    positions = pickle.load(f)

In [3]:
len(positions)

1400

In [16]:
positions[0:10]

[[-2400.0, -500.0],
 [-2316.5400009155273, -500.0],
 [-2233.0800018310547, -500.0],
 [-2149.620002746582, -500.0],
 [-2066.1600036621094, -500.0],
 [-1982.7000045776367, -500.0],
 [-1899.240005493164, -500.0],
 [-1815.7800064086914, -500.0],
 [-1732.3200073242188, -500.0],
 [-1648.860008239746, -500.0]]

In [10]:
oni_params = {
    "cropping": {
        "top-left": [130, 5],
        "top-right": [140, 595],
        "width": 428,
        "height": 684
    },
    "safe_focus": {
        "upper_thresh": -300,
        "lower_thresh": -470,
        "af_intensity_thresh":1500
    },
    "microscope_name": "60xAALZ7W2T",
    "xy_positions_mm": positions,
    "z_relative_positions_um": [],
    "light_program":[{'laser_power': [10.0, 0.0, 0.0, 0.0], 'exposure_ms': 75}, # ordered 405, 488, 561, 640
                     {'laser_power': [0.0, 10.0, 0.0, 0.0], 'exposure_ms': 75},
                     {'laser_power': [0.0, 0.0, 10.0, 0.0], 'exposure_ms': 150},
                     {'laser_power': [0.0, 0.0, 0.0, 10.0], 'exposure_ms': 150}
                     ],
    "start_z": -456,
    "num_z_slices": 20,
    "relative_top_z_um": 3.0,
    "relative_bottom_z_um": -3.0,
    'fovIncrement_mm': {'x': 0.08345999908447266,'y': 0.13337998962402345}, # ONI default with no overlap
    'save_destination': "C:/Data/DEFAULT_USER/",
    'autofocus_interval':1,
    'autofocus_points':5,
    'max_af_search': 25
}

def _make_tiles(starting_pos,n_x,n_y,x_inc_um,y_inc_um):
        x_start = starting_pos['x']
        y_start = starting_pos['y']
        positions = []
        xs = list(range(0,n_x))
        ys = list(range(0,n_y))
        if (len(ys) % 2) == 0:
            all_xs = (xs + list(reversed(xs)))*floor(len(ys)/2)
        else:
            all_xs = (xs + list(reversed(xs)))*floor(len(ys)/2)
            all_xs += xs
        all_ys = []
        for y in ys:
            all_ys += [y]*len(xs)
        tmp_positions = list(zip(all_xs,all_ys))
        positions = []
        for pos in tmp_positions:
            positions.append([x_start + (pos[0] * x_inc_um), y_start + (pos[1] * y_inc_um)])
        return positions

# custom_positions = None
def position_generator(oni_params,custom_positions,starting_xy_um={'x':0,'y':0},tile_num_x=1,tile_num_y=1,overlap=0.0):
    if custom_positions is not None:
        oni_params['xy_positions_mm'] = custom_positions
    else:
        x_inc_um = (oni_params['fovIncrement_mm']['x'] * 1000) * (1 - overlap)
        y_inc_um = (oni_params['fovIncrement_mm']['y'] * 1000) * (1 - overlap)
        oni_params['xy_positions_mm'] = _make_tiles(starting_xy_um,tile_num_x,tile_num_y,x_inc_um,y_inc_um)
    # generate relative_zs
    oni_params['z_relative_positions_um'] = np.linspace(oni_params['relative_bottom_z_um'],oni_params['relative_top_z_um'],oni_params['num_z_slices']).tolist()
    return oni_params

In [11]:
oni_params = position_generator(oni_params,positions)

In [12]:
len(oni_params['xy_positions_mm'])

1400

In [13]:
with open('oni_params.pkl', 'wb') as f:
    pickle.dump(oni_params, f)

In [ ]:
final_positions_idx = list(np.unique(np.where(rois > 0, 1, 0) * fov_label_tensor))

In [61]:
oni_params

{'cropping': {'top-left': [130, 5],
  'top-right': [140, 595],
  'width': 428,
  'height': 684},
 'safe_focus': {'upper_thresh': -300,
  'lower_thresh': -500,
  'af_intensity_thresh': 2000},
 'microscope_name': '60xAALZ7W2T',
 'xy_positions_mm': [[0.0, 0.0],
  [75.11399917602539, 0.0],
  [150.22799835205078, 0.0],
  [225.34199752807615, 0.0],
  [300.45599670410155, 0.0],
  [375.56999588012695, 0.0],
  [450.6839950561523, 0.0],
  [525.7979942321778, 0.0],
  [600.9119934082031, 0.0],
  [676.0259925842284, 0.0],
  [751.1399917602539, 0.0],
  [826.2539909362793, 0.0],
  [826.2539909362793, 120.0419906616211],
  [751.1399917602539, 120.0419906616211],
  [676.0259925842284, 120.0419906616211],
  [600.9119934082031, 120.0419906616211],
  [525.7979942321778, 120.0419906616211],
  [450.6839950561523, 120.0419906616211],
  [375.56999588012695, 120.0419906616211],
  [300.45599670410155, 120.0419906616211],
  [225.34199752807615, 120.0419906616211],
  [150.22799835205078, 120.0419906616211],
  [75

In [37]:
fluids

,NodeID,FluidType,BufferName,LoadVolume
0,ot2,source,ot2,3
1,2,source,wash buffer,40
2,3,source,imaging buffer,21
3,4,source,tcep,20
4,5,source,2x-SSC,50
5,6,source,universal_fluors,15
6,7,source,NaN,0
7,8,source,NaN,0
8,waste,destination,waste_tank,0


In [38]:
def get_fluid_name(nodeID):
    return fluids[fluids['NodeID']==nodeID]['BufferName'].iloc[0]

In [39]:
reagent_vols = {}
for i in experiment.keys():
    if experiment[i]['step_type'] == 'fluid':
        buffer = get_fluid_name(experiment[i]['step_metadata']['fluid'])
        if buffer not in reagent_vols:
            reagent_vols[buffer] = 0
        reagent_vols[buffer] += experiment[i]['step_metadata']['volume']

In [40]:
reagent_vols

{'ot2': 15.0,
 'wash buffer': 40,
 'universal_fluors': 15.0,
 'imaging buffer': 15.0,
 'tcep': 18,
 '2x-SSC': 45}

In [42]:
for i in experiment.keys():
    if experiment[i]['step_type'] == 'liquid_handler':
        print(experiment[i]['step_metadata']['info']['location'])

A1
A2
A3
A4
A5
A6
A7
A8
A9
A10
